<h1>Image Recolouring Project</h1>

Project developed by Alejandro Cano Caldero and Jesús Moncada Ramírez for the subject Neural Networks and Deep Learning, University of Padova, 2022-23.


In [15]:
from PIL import Image

import numpy as np

import torch

from torchvision import transforms, datasets
from torchvision.transforms import transforms

from torch.utils.data import DataLoader, Dataset  

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as op

In [ ]:
# Define the execution device 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<h2>1. Dataset</h2>

For the dataset we have used [ImageNette](https://github.com/fastai/imagenette), a reduced version of ImageNet, specifically the fill size images version.

In [16]:
class ImageDataset(Dataset):
    def __init__(self, image_path, transform=None):
        super(ImageDataset, self).__init__()
        self.data = datasets.ImageFolder(image_path,  transform)

    def __getitem__(self, idx):
        x, y = self.data[idx]
        return x, y

    def __len__(self):
        return len(self.data)

In [19]:
img_path = 'D:/ImageColoringDataset' # TODO: route in Google Drive

tsfm = transforms.Compose([
        transforms.ToTensor()
])

data = ImageDataset(img_path, tsfm) # create the dataset

FileNotFoundError: ignored

In [ ]:
# Define a class to convert colored images into grayscale images

class GrayScaleImage():
    
    def __init__(self, images):
        self.images = images
    
    def dataset2gray(self):
        return map(lambda img: img.convert('L'), self.images)

In [ ]:
# Test GrayScaleImage class

img_dir = "C:/Alejandro/Fotos/meme2.png"
img_rgb = Image.open(img_dir)
#img_rgb.show()

images = [img_rgb]
test = GrayScaleImage(images)
img_gray = test.dataset2gray()

for i in img_gray:
    i.show()

FileNotFoundError: ignored

In [ ]:
# Define normalization

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
# Load the data

# Use GrayScaleImage() to get the black-and-white images

# Gather all the data

# Normalize it
    

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, in_channels, filters, kernel_size, stride=1):
        super().__init__()
        
        self.layer1 = nn.Conv2d(in_channels, out_channels=64, kernel_size=kernel_side, stride=2, padding="same")
        self.layer2 = nn.Conv2d(64, out_channels=128, kernel_size=kernel_side, stride=2, padding="same")
        self.layer2_bn = nn.BatchNorm2d(128)
        self.layer3 = nn.Conv2d(128, out_channels=256, kernel_size=kernel_side, stride=2, padding="same")
        self.layer3_bn = nn.BatchNorm2d(256)
        self.layer4 = nn.Conv2d(256, out_channels=512, kernel_size=kernel_side, padding="same") # stride = 1
        self.layer4_bn = nn.BatchNorm2d(512)
        self.layer5 = nn.Conv2d(512, out_channels=1, kernel_size=kernel_side, padding="same")
    
    def forward(self, x):
        d = F.LeakyReLU(self.layer1(x), 0.2)
        d = F.LeakyReLU(self.layer2_bn(self.layer2(d)), 0.2)
        d = F.LeakyReLU(self.layer3_bn(self.layer3(d)), 0.2)
        d = F.LeakyReLU(self.layer4_bn(self.layer4(d)), 0.2)
        d = self.layer5(d)
        
        return F.Tanh(d) # Not sure about this activation function (sigmoid??)
        

# model = Discriminator()
# model.to(device)

# loss_fn = nn.BCELoss(weight=torch.tensor(0.5))
# optimizer = op.Adam(model.parameters(), lr=0.0002, weight_decay=0.5)

In [ ]:
class Generator(nn.Module):
  def __init__():
    pass
  def forward():
    pass